# Day 09: Movie Theater

## Part 1

In [1]:
with open('input.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

In [2]:
areas = {}
for i in data:
    for j in data:
        if(i == j or any([pair in areas.keys() for pair in [(i,j), (j,i)]])):
            continue
        else:
            x_vals = [int(i.split(',')[0]), int(j.split(',')[0])]
            y_vals = [int(i.split(',')[1]), int(j.split(',')[1])]
            area = (abs(max(x_vals)-min(x_vals))+1)*(abs(max(y_vals)-min(y_vals))+1)
            areas[(i,j)] = area

# print(max(areas.values()))

## Part 2

In [3]:
import pandas as pd

### Helper functions for visualizing grid

In [4]:
# some helper functions for visualizing problem
def reset_grid(size, bounds, default_val='.', bound_val='#'):
    grid = create_empty_grid(size, default_val)
    for bound in bounds:
        grid = write_to_grid(bound, grid, bound_val)
    return grid

def create_empty_grid(size, val='.'):
    return [[val for i in range(size)] for i in range(size)]

def write_point_to_grid(point, grid, char='@'):
    grid[point[0]][point[1]] = char
    return grid

def write_to_grid(bound, grid, char='@'):
    if(bound[0][0] != bound[1][0]):
        for i in range(min(bound[0][0], bound[1][0]), max(bound[0][0], bound[1][0])+1):
            grid[i][bound[0][1]] = char
    elif(bound[0][1] != bound[1][1]):
        for i in range(min(bound[0][1], bound[1][1]), max(bound[0][1], bound[1][1])+1):
            grid[bound[0][0]][i] = char
    return grid

### Area and perimeter

In [5]:
def get_perimeter(point_a, point_b):
    points = set()
    for i in range(min([point_a[0], point_b[0]]), max([point_a[0], point_b[0]])+1):
        for p in [(i, point_a[1]), (i, point_b[1])]:
            if(p not in points):
                points.add(p)

    for i in range(min([point_a[1], point_b[1]]), max([point_a[1], point_b[1]])+1):
        for p in [(point_a[0], i), (point_b[0], i)]:
            if(p not in points):
                points.add(p)
    return points

def get_area(point_a, point_b):
    x_vals = [int(point_a[0]), int(point_b[0])]
    y_vals = [int(point_a[1]), int(point_b[1])]
    return (abs(max(x_vals)-min(x_vals))+1)*(abs(max(y_vals)-min(y_vals))+1)

### Getting outer perimeter

In [ ]:
def get_next_direction(cur_direction, cur_move, next_move):
    # get the direction of the outer edge in format [row_change, col_change] from point based on current traversal direction and next traversal direction
    # e.x. if the outer edge is on the left, we are currently traversing the grid upwards, and the next traversal direction is to the right, the outer edge becomes the top

    """
    likely unhelpful diagram:
    d1 is outer edge during m1 movement direction
    d2 is outer edge during m2 movement direction
    this changes based on angle and direction
                d2
                |
         -------m2------->
        ^
        |
    d1--m1
        |
        |
    """

    # if move direction doesn't change, neither does outer edge direction
    if(cur_move == next_move):
        return cur_direction
    
    # outer edge is on the right if:
    # cur_dir = [1, 0], cur_move = [0, 1], next_move = [-1, 0], sum = 1
    # cur_dir = [1, 0], cur_move = [0, -1], next_move = [1, 0], sum = 1
    # cur_dir = [-1, 0], cur_move = [0, 1], next_move = [1, 0], sum = 1
    # cur_dir = [-1, 0], cur_move = [0, -1], next_move = [-1, 0], sum = -3
    elif(cur_direction[0]+cur_move[1]+next_move[0] in (1, -3)):
        return [0, 1]
    
    # outer edge is on the left if:
    # cur_dir = [1, 0], cur_move = [0, 1], next_move = [1, 0], sum = 3
    # cur_dir = [1, 0], cur_move = [0, -1], next_move = [-1, 0], sum = -1
    # cur_dir = [-1, 0], cur_move = [0, 1], next_move = [-1, 0], sum = -1
    # cur_dir = [-1, 0], cur_move = [0, -1], next_move = [1, 0], sum = -1
    elif(cur_direction[0]+cur_move[1]+next_move[0] in (-1, 3)):
        return [0, -1]
    
    # outer edge is on the bottom if:
    # cur_dir = [0, 1], cur_move = [1, 0], next_move = [0, -1], sum = 1
    # cur_dir = [0, 1], cur_move = [-1, 0], next_move = [0, 1], sum = 1
    # cur_dir = [0, -1], cur_move = [1, 0], next_move = [0, 1], sum = 1
    # cur_dir = [0, -1], cur_move = [-1, 0], next_move = [0, -1], sum = -3
    elif(cur_direction[1]+cur_move[0]+next_move[1] in (1, -3)):
        return [1, 0]
    
    # outer edge is on the top if:
    # cur_dir = [0, 1], cur_move = [1, 0], next_move = [0, 1], sum = 3
    # cur_dir = [0, 1], cur_move = [-1, 0], next_move = [0, -1], sum = -1
    # cur_dir = [0, -1], cur_move = [1, 0], next_move = [0, -1], sum = -1
    # cur_dir = [0, -1], cur_move = [-1, 0], next_move = [0, 1], sum = -1
    elif(cur_direction[1]+cur_move[0]+next_move[1] in (-1, 3)):
        return [-1, 0]

def get_outer_perim(bounds, start=None, start_direction=None):
    # return all points directly outside of the shape
    global bounds_points
    global invalid_directions

    start_index = 0 if start is None else bounds.index(start)
    # this represents the values that need to be added to point[0], point[1] to reach the adjacent point that is outside the shape
    # e.x. point [4, 3] is along leftmost edge of the shape, so its adjacent outside point is [4, 2] -> [4+cur_direction[0], 3+cur_direction[1]] = [4+0, 3+(-1)] = [4, 2]
    cur_direction = start_direction if start_direction is not None else [0, -1] # this actually only has a 1 in 4 shot of working if start_direction isn't supplied but i can't be bothered to build out the logic

    # points along outside edge
    invalid_points = set()

    # for each bound (2 points connected by straight vertical or horizontal line)
    for i in range(len(bounds)):
        # current index is start index + i, but loops back to 0 when overflowing
        cur_index = (start_index+i)%len(bounds)

        edge = bounds[cur_index]
        
        # moving along y axis
        if(edge[0][0] != edge[1][0]):
            # next move direction is up if point 1 < point 2, otherwise down
            next_move = [1 if edge[0][0] < edge[1][0] else -1, 0]
            
            # get next direction of outside edge
            if(cur_index != start_index):
                cur_direction = get_next_direction(cur_direction, move, next_move)

            move = next_move

        # move along x axis
        elif(edge[0][1] != edge[1][1]):
            next_move = [0, 1 if edge[0][1] < edge[1][1] else -1]

            # get next direction of outside edge
            if(cur_index != start_index):
                cur_direction = get_next_direction(cur_direction, move, next_move)

            move = next_move
        
        # if moving along y axis
        if(move[0] != 0):
            # for each point from start of line to end of line in direction provided
            for i in range(edge[0][0], edge[1][0]+move[0], move[0]):
                bound_point = (i, edge[0][1])
                outer_point = (i, edge[0][1]+cur_direction[1])

                bounds_points.add(bound_point)

                # remove original shape perimeter points from invalid list
                if(bound_point in invalid_points):
                    invalid_points.remove(bound_point)

                # add invalid points only if not also included in original shape perimeter
                if(outer_point not in bounds_points):
                    invalid_points.add(outer_point)

        # if moving along x axis 
        elif(move[1] != 0):
            # for each point from start of line to end of line in direction provided
            for i in range(edge[0][1], edge[1][1]+move[1], move[1]):
                bound_point = ((edge[0][0], i))
                outer_point = (edge[0][0]+cur_direction[0], i)

                bounds_points.add(bound_point)

                # remove original shape perimeter points from invalid list
                if(bound_point in invalid_points):
                    invalid_points.remove(bound_point)
                    
                # add invalid points only if not also included in original shape perimeter
                if(outer_point not in bounds_points):
                    invalid_points.add(outer_point)

    return invalid_points

### DataFrames!

In [ ]:
def create_invalid_table(invalid_points):
    # turn list of invalid points into individual rows where the x and y value of the points are separated into row and col columns
    return pd.DataFrame({'row':[i[0] for i in invalid_points], 'col':[i[1] for i in invalid_points]}).sort_values(by=['row','col'],ignore_index=True,inplace=False)

def create_min_valid_table(points, invalid_table):
    # for each point, find the closest invalid point above, below, to the left of, and to the right of it
    min_valid_points = {'point':[], 'row_lower':[], 'row_upper':[], 'col_lower':[], 'col_upper':[]}

    for point in points:
        min_valid_points['point'].append(point)

        # upper and lower column bounds 
        min_valid_points['col_lower'].append(max(invalid_table.query(f'row=={point[0]} & col < {point[1]}')['col']))
        min_valid_points['col_upper'].append(min(invalid_table.query(f'row=={point[0]} & col > {point[1]}')['col']))

        # upper and lower row bounds
        min_valid_points['row_lower'].append(max(invalid_table.query(f'col=={point[1]} & row < {point[0]}')['row']))
        min_valid_points['row_upper'].append(min(invalid_table.query(f'col=={point[1]} & row > {point[0]}')['row']))
    
    # set index as point tuple to speed up data retreival
    return pd.DataFrame(min_valid_points).set_index('point')

### Run calculations

In [9]:
# each sequential pair of points
bounds = []
for i in range(len(data)):
    point_a = tuple([int(val) for val in data[i-1].split(',')])
    point_b = tuple([int(val) for val in data[i].split(',')])

    bounds.append((point_a, point_b))

# find pair with minimum x value where x value does not change
leftmost_bound = min(bounds, key = lambda x: min([x[0][1], x[1][1]]) if x[0][1] == x[1][1] else 100000)

# setting up global vars
outer_perim = set()
bounds_points = set()

# getting collection of points that make up perimeter around shape
invalid_points = get_outer_perim(bounds, start=leftmost_bound, start_direction=[0, -1])

# setting up dataframes
invalid_points_table = create_invalid_table(invalid_points)
point_bounds = create_min_valid_table([(int(row.split(',')[0]), int(row.split(',')[1])) for row in data],invalid_points_table)

# setting up area vars
max_area = 0
max_area_pair = None

for i in data:
    # tuple of ints for current point
    point_a = (int(i.split(',')[0]), int(i.split(',')[1]))

    # upper and lower row and col bounds for current point
    a_bounds = point_bounds.loc[[point_a]].iloc[0].to_dict()

    # for each point, loop through each point to collect all pairs
    for j in data:
        # skip when point is the same
        if(i == j):
            continue

        # start assuming pair forms valid rectangle
        valid = 1

        # tuple of ints for comparison point
        point_b = (int(j.split(',')[0]), int(j.split(',')[1]))

        # other corners formed by rectangle with point_a and point_b as opposite corners
        corners = [
            (point_a[0], point_b[1])
            ,(point_b[0], point_a[1])
            ]
        
        # for each corner, if it exceeds the valid bounds of point_a, it is invalid
        for corner in corners:
            if(corner[0] == point_a[0] and (corner[1] >= a_bounds['col_upper'] or corner[1] <= a_bounds['col_lower'])):
                valid = 0
                break
            elif(corner[1] == point_a[1] and (corner[0] >= a_bounds['row_upper'] or corner[0] <= a_bounds['row_lower'])):
                valid = 0
                break
        
        # if invalid, move on
        if(not valid):
            continue

        # upper and lower row and col bounds for second point
        b_bounds = point_bounds.loc[[point_b]].iloc[0].to_dict()
        # for each corner, if it exceeds the valid bounds of point_b, it is invalid
        for corner in corners:
            if(corner[0] == point_b[0] and (corner[1] >= b_bounds['col_upper'] or corner[1] <= b_bounds['col_lower'])):
                valid = 0
                break
            elif(corner[1] == point_b[1] and (corner[0] >= b_bounds['row_upper'] or corner[0] <= b_bounds['row_lower'])):
                valid = 0
                break
        
        # if invalid, move on
        if(not valid):
            continue
        
        # if point still valid and area is greater than current max, area is new max and current pair is max pair
        area = get_area(point_a, point_b)
        if(valid and area > max_area):
            max_area = area
            max_area_pair = (point_a, point_b)

# print(max_area_pair, max_area)